<a href="https://colab.research.google.com/github/BeratE/GoogleColab/blob/main/PDFRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependency Management

In [ ]:
# Install utilities
!apt-get install poppler-utils tesseract-ocr

In [ ]:
# Install dependencies
!pip install langchain huggingface_hub sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference] -q
!pip install torch==2.2.1 wrapt==1.11.0 pillow==10.1.0
# Session Restart required because python dependency management is a mess!

# Language Model

In [1]:
import os
import getpass
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(
        "Please provide your HuggingFaceHub API Token: ")

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1)

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Document Embedding

In [ ]:
from pathlib import Path
from langchain.document_loaders import UnstructuredPDFLoader
root = "/content/gdrive/My Drive/Books/GameAIPro"
loaders = [UnstructuredPDFLoader(str(fn)) for fn in Path(root).glob('**/*.pdf')]
loaders

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

In [6]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=index.vectorstore.as_retriever(),
                                    input_key="question")

# QA

In [13]:
import textwrap

def wrap_text(text, width=110):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

In [20]:
prompt = input('How can I help you?\n\n')
while (prompt != "quit"):
  answer = chain.invoke(prompt)
  print("\n")
  print(wrap_text(answer["result"]))
  prompt = input("\n")

How can I help you?

What is the best strategy to implement enemy AI in a simple fighting game?


 The best strategy to implement enemy AI in a simple fighting game depends on the specific requirements and
constraints of the game. If the battles are relatively simple and the NPC faces uniform challenges, static
analysis approaches such as crafting rules or building decision trees can be sufficient. However, these
approaches can be brittle when the NPC is faced with new or unforeseen situations. Alternatively, simulating
battles multiple times and recording outcomes can provide an estimate of the NPC's chances of winning. More
advanced approaches, such as using RL algorithms to fine-tune AI behavior, can adapt to make NPCs more
competitive or collaborative, but require more game knowledge and coding effort. Ultimately, the choice of
strategy will depend on the resources available and the desired level of AI complexity.

quit
